In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot

In [3]:
new_model = tf.keras.models.load_model("D:/Modelos/EfficientNet_B1_IMAGENET_IC_SCM3")

In [4]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb1 (Functional)  (None, 1, 1, 1280)        6575239   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 4)                 5124      
Total params: 6,580,363
Trainable params: 6,518,308
Non-trainable params: 62,055
_________________________________________________________________


In [5]:
#carrega os arrays de imagens e de classes
train_X = np.load("D:/Displasia/displasiaDataAug/displasiaOriginalDataX_teste3.npy")
train_label = np.load("D:/Displasia/displasiaDataAug/displasiaOriginalDataYOneHot_teste3.npy")
XSCM = np.load("D:/Displasia/displasiaDataAug/displasiaOriginalDataX_teste3_SCM.npy")
#dataX = np.load("Image Arrays/displasiaGeneratedDataX.npy")
#dataY = np.load("Image Arrays/displasiaGeneratedDataY.npy")

#shape dos arrays
train_X.shape, train_label.shape

((75776, 32, 32, 3), (75776, 4))

In [6]:
#from sklearn.model_selection import train_test_split

#divide os dados em 80% para treino e 20% para teste
#train_X, test_X, train_label, test_label = train_test_split(X, Y, test_size=0.2, random_state=13)

#divide os dados de treino em 80% para treino e 20% para validação
#train_X, valid_X, train_label, valid_label = train_test_split(train_X, train_label, test_size=0.2, random_state=13)

#mostra os shapes resultantes das divisões
#print("Training:   ", train_X.shape, train_label.shape)
#print("Validation: ", valid_X.shape, valid_label.shape)
#print("Testing:    ", test_X.shape, test_label.shape)

In [7]:
#divide os dados em 80% para treino e 20% para teste
#train_X_SCM, test_X_SCM, train_label_SCM, test_label_SCM = train_test_split(XSCM, Y, test_size=0.2, random_state=13)

#divide os dados de treino em 80% para treino e 20% para validação
#train_X_SCM, valid_X_SCM, train_label_SCM, valid_label_SCM = train_test_split(train_X_SCM, train_label_SCM, test_size=0.2, random_state=13)

#mostra os shapes resultantes das divisões
#print("Training:   ", train_X_SCM.shape, train_label_SCM.shape)
#print("Validation: ", valid_X.shape, valid_label.shape)
#print("Testing:    ", test_X_SCM.shape, test_label_SCM.shape)

In [8]:
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import keras
from sklearn.model_selection import KFold
#kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=7)
kfold = KFold(n_splits=5, shuffle=True, random_state=7)
cvscores = []

mean = [[[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]]]

std = [[[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]],
         [[],[],[],[],[]]]

for train, test in kfold.split (train_X, train_label):

    intermediate_layer_model = keras.Model(inputs=new_model.input,
                                           outputs=new_model.get_layer('flatten').output)
    intermediate_output = intermediate_layer_model.predict(train_X[train])

    newArr = np.append(intermediate_output, XSCM[train] , axis=1)

    clf=RandomForestClassifier(n_estimators=100)
    #train_labels2=train_labels[:49984]
    clf.fit(newArr,train_label[train])

    intermediate_layer_model_2 = keras.Model(inputs=new_model.input,
                                           outputs=new_model.get_layer('flatten').output)
    intermediate_output_2 = intermediate_layer_model_2.predict(train_X[test])

    newArr_test = np.append(intermediate_output_2, XSCM[test] , axis=1)

    y_pred=clf.predict(newArr_test)

    # Model Accuracy, how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(train_label[test], y_pred))
    cvscores.append(metrics.accuracy_score(train_label[test], y_pred)*100)
    
    from keras.utils import to_categorical

    predicted_classes = y_pred

    predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

    predicted_classes = to_categorical(predicted_classes)

    
    num_classes = 4
    from sklearn.metrics import classification_report
    target_names = ["Class {}".format(i) for i in range(num_classes)]
    print(classification_report(train_label[test], predicted_classes, target_names=target_names))
    
    from sklearn.metrics import confusion_matrix

    #converte os arrays de classe de one-hot para númerico
    test_label_aux = np.argmax(train_label[test], axis=1)
    predicted_classes = np.argmax(predicted_classes, axis=1)

    #gera a matriz de confusão a partir da predição feita
    cm = confusion_matrix(test_label_aux, predicted_classes)

    #gera os valores de falso positivo, falso negativo, verdadeiro positivo e verdadeiro negativo
    fp = cm.sum(axis=0) - np.diag(cm)  
    fn = cm.sum(axis=1) - np.diag(cm)
    tp = np.diag(cm)
    tn = cm.sum() - (fp + fn + tp)

    #gera a sensitividade, especificidade, acurácia e precisão
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy    = (tp + tn) / (tp + tn + fp + fn)
    precision   = tp / (tp + fp)
    f1_score = 2/((1/precision)+(1/sensitivity))

    #mostra os resultados
    print("Precision:  ", precision)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)
    print("F1-score:   ", f1_score)
    print("Accuracy:   ", accuracy)
    
    x = []
    x.append(precision)
    x.append(sensitivity)
    x.append(specificity)
    x.append(f1_score)
    x.append(accuracy)
    
    x = np.asarray(x)
    
    x = x.transpose()
    print()
    print(x)
    
    for i in range(4):
        for j in range(5):
            mean[i][j].append(x[i][j])
            std[i][j].append(x[i][j])
    
print("%.2f%% (+/- %.2f%%)" %(np.mean(cvscores),np.std(cvscores)))


Accuracy: 0.9984164687252574
              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00      3867
     Class 1       1.00      1.00      1.00      3755
     Class 2       1.00      1.00      1.00      3686
     Class 3       1.00      1.00      1.00      3848

   micro avg       1.00      1.00      1.00     15156
   macro avg       1.00      1.00      1.00     15156
weighted avg       1.00      1.00      1.00     15156
 samples avg       1.00      1.00      1.00     15156

Precision:   [0.99948307 0.99680596 0.99755767 0.99973992]
Sensitivity: [1.         0.99733688 0.99728703 0.9989605 ]
Specificity: [0.99982284 0.99894746 0.99921534 0.99991157]
F1-score:    [0.99974147 0.99707135 0.99742233 0.99935006]
Accuracy:    [0.99986804 0.99854843 0.99874637 0.9996701 ]

[[0.99948307 1.         0.99982284 0.99974147 0.99986804]
 [0.99680596 0.99733688 0.99894746 0.99707135 0.99854843]
 [0.99755767 0.99728703 0.99921534 0.99742233 0.99874637]
 [0.99973992

In [9]:
mean = np.asarray(mean)
std = np.asarray(std)
for i in mean:
    print(np.mean(i,axis=1))
print()
for i in std:
    print(np.std(i,axis=1))

[0.99952759 0.99978478 0.99984134 0.99965613 0.99982844]
[0.99662619 0.996782   0.99887344 0.99670394 0.9983504 ]
[0.99720405 0.99741171 0.99906722 0.99730771 0.99865393]
[0.99914985 0.99852014 0.99971898 0.99883488 0.99941934]

[1.90516758e-04 2.03813158e-04 6.66057618e-05 6.82112794e-05
 3.23279627e-05]
[0.00071525 0.00033031 0.00024718 0.00040873 0.00021687]
[0.00062324 0.00054079 0.00021425 0.00042571 0.00021524]
[0.00036345 0.00027533 0.00011536 0.00030575 0.00014696]
